# Slalom turns

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot


pyplot.style.use('seaborn')

In [ ]:
from trajectory import Line
from trajectory import lines_intersection

In [ ]:
from dataclasses import dataclass

from matplotlib.patches import Rectangle
from numpy import array
from numpy import cos
from numpy import pi
from numpy import sin
from numpy import sign
from numpy import sqrt
from numpy import tan
from numpy.linalg import norm
from pandas import DataFrame


@dataclass
class Maze:
    cell: float
    post: float
    

@dataclass
class RobotPhysics:
    mass: float
    moment_of_inertia: float
    wheels_separation: float
    max_angular_velocity: float


class TurnProfile:
    def __init__(self, profile):
        self.profile = profile

    def plot_forces(self):
        self.profile[['centrifugal_force', 'angular_acceleration_force', 'total_force']].plot(style='.-')


class SlalomTurnProfile(TurnProfile):
    def __init__(self, profile, entry, exit, maze):
        super().__init__(profile)
        self.entry = entry
        self.exit = exit
        self.maze = maze

    def describe(self):
        print('Turn completed in %.3f seconds' % self.profile.index[-1])
        print('Finished at (%.5f, %.5f)' % tuple(self.profile[['x', 'y']].iloc[-1].values))
        step = self.profile['linear_velocity'].iloc[0] * self.profile['period'].iloc[0]
        before = norm(self.profile[['x_shift', 'y_shift']].iloc[0].values -
                      self.entry.reference)
        before *= sign(norm(self.profile[['x_shift', 'y_shift']].iloc[1].values -
                            self.entry.reference) - before)
        after = norm(self.profile[['x_shift', 'y_shift']].iloc[-1].values -
                     self.exit.reference)
        after *= sign(norm(self.profile[['x_shift', 'y_shift']].iloc[-2].values -
                           self.exit.reference) - after)
        radius = self.profile['radius'].iloc[0]
        transition = (~self.profile['arc']).sum() * step / 2
        arc = self.profile['arc'].sum() * step
        print('Parameters: {%.5f, %.5f, %.5f, %.5f, %.5f, xxx}' % (before, after, radius, transition, arc))

    def plot_trajectory(self):
        figure, axes = pyplot.subplots(1, figsize=(8, 8), subplot_kw={'aspect': 'equal'})
        post = self.maze.post
        axes.add_patch(Rectangle((-post / 2, -post / 2), post, post))
        axes.plot(self.profile['x_shift'], self.profile['y_shift'], '.-')
        pyplot.show()


def turn_shift(entry, exit, profile):
    expected = lines_intersection(entry, exit)
    start = Line(**profile[['x', 'y']].iloc[0].to_dict(), angle=entry.angle)
    stop = Line(**profile[['x', 'y']].iloc[-1].to_dict(), angle=exit.angle)
    actual = lines_intersection(start, stop)
    return expected - actual


def turn_profile(angle, max_angular_velocity,
                 max_angular_acceleration, time_period):
    duration = max_angular_velocity / max_angular_acceleration * pi
    transition_angle = duration * max_angular_velocity / pi
    arc = (abs(angle) - 2 * transition_angle) / max_angular_velocity
    transition = duration / 2
    max_angular_velocity = max_angular_velocity * sign(angle)

    angular_velocity = [0]
    time = [0]
    while True:
        if time[-1] >= 2 * transition + arc:
            break
        new_angular_velocity = max_angular_velocity
        if time[-1] < transition:
            factor = time[-1] / transition
            new_angular_velocity *= sin(factor * pi / 2)
        elif time[-1] >= transition + arc:
            factor = (time[-1] - arc) / transition
            new_angular_velocity *= sin(factor * pi / 2)
        angular_velocity.append(new_angular_velocity)
        time.append(time[-1] + time_period)

    profile = DataFrame({'angular_velocity': angular_velocity})
    profile['arc'] = profile['angular_velocity'] == max_angular_velocity
    return profile


def complete_profile(profile, angle, radius, linear_velocity, robot, time_period):
    profile['radius'] = radius
    profile['period'] = time_period
    profile['linear_velocity'] = linear_velocity
    profile['angle'] = (profile['angular_velocity'] * profile['period']).cumsum() + angle
    profile['x'] = (profile['linear_velocity'] * cos(profile['angle']) * profile['period']).cumsum()
    profile['y'] = (profile['linear_velocity'] * sin(profile['angle']) * profile['period']).cumsum()
    profile['centrifugal_force'] = robot.mass * profile['linear_velocity'] * profile['angular_velocity'] / 2
    angular_acceleration = profile['angular_velocity'].diff() / profile['period']
    profile['angular_acceleration_force'] = \
        (robot.moment_of_inertia * angular_acceleration / robot.wheels_separation).abs()
    profile['total_force'] = sqrt(profile['centrifugal_force'] ** 2 + profile['angular_acceleration_force'] ** 2)
    profile['time'] = profile['period'].cumsum()
    profile = profile.set_index('time')
    return profile


class Simulator:
    def __init__(self, robot, maze, time_period):
        self.robot = robot
        self.maze = maze
        self.time_period = time_period
    
    def inplace(self, angle, force=0.25):
        angular_acceleration = force * self.robot.wheels_separation / self.robot.moment_of_inertia
        max_angular_velocity = sqrt(abs(angle) / 2 * angular_acceleration)
        max_angular_velocity = min(max_angular_velocity,
                                   self.robot.max_angular_velocity)
        profile = turn_profile(angle, max_angular_velocity,
                               angular_acceleration, self.time_period)
        profile = complete_profile(profile, 0., 0., 0., self.robot, self.time_period)
        return TurnProfile(profile)

    def slalom(self, entry, exit, radius, force=0.25, shift=None):
        x, y, angle = entry
        entry = Line(x=x*self.maze.cell, y=y*self.maze.cell, angle=angle)
        x, y, angle = exit
        exit = Line(x=x*self.maze.cell, y=y*self.maze.cell, angle=angle)
        angle = exit.angle - entry.angle
        linear_velocity = sqrt(2 * force * radius / self.robot.mass)
        max_angular_velocity = linear_velocity / radius
        max_angular_velocity = min(max_angular_velocity,
                                   self.robot.max_angular_velocity)
        max_angular_acceleration = force * self.robot.wheels_separation / self.robot.moment_of_inertia
        max_angular_velocity_transition = sqrt(abs(angle) / 2 * max_angular_acceleration)
        if max_angular_velocity_transition < max_angular_velocity:
            raise ValueError
        profile = turn_profile(angle, max_angular_velocity,
                               max_angular_acceleration, self.time_period)
        profile = complete_profile(profile, entry.angle, radius, linear_velocity, self.robot, self.time_period)
        if shift is None:
            shift = turn_shift(entry, exit, profile)
        else:
            shift += entry.reference
        profile['x_shift'] = profile['x'] + shift[0]
        profile['y_shift'] = profile['y'] +  shift[1]
        return SlalomTurnProfile(profile, entry, exit, self.maze)

In [ ]:
classic = Maze(cell=0.18, post=0.012)
half = Maze(cell=0.09, post=0.006)

bulebule = RobotPhysics(
    mass=0.110,
    moment_of_inertia=(0.110 * 0.05 ** 2) / 2,
    wheels_separation=0.065,
    max_angular_velocity=20,
)

In [ ]:
simulate = Simulator(robot=bulebule, maze=classic, time_period=0.00001)

## Static turns

In [ ]:
profile = simulate.inplace(angle=pi)
profile.plot_forces()

## Search turn

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, 0),
    exit=(.5, 0, pi/2),
    radius=0.04921,
)
profile.describe()
profile.plot_trajectory()

## 90º turn

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, 0),
    exit=(.5, 0, pi/2),
    radius=0.1172,
)
profile.describe()
profile.plot_trajectory()

## 180º turn

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, 0),
    exit=(0, .5, pi),
    radius=0.08882,
    shift=array((-0.05, 0))
)
profile.describe()
profile.plot_trajectory()

## Turn to 45º

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, 0),
    exit=(.5, 0, pi/4),
    radius=0.12,
)
profile.describe()
profile.plot_trajectory()

## Turn to 135º

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, 0),
    exit=(0, .5, 3/4*pi),
    radius=0.0767,
)
profile.describe()
profile.plot_trajectory()

## Turn diagonal 90º

In [ ]:
profile = simulate.slalom(
    entry=(0, -.5, pi/4),
    exit=(0, .5, 3/4*pi),
    radius=0.08,
)
profile.describe()
profile.plot_trajectory()